In [3]:
%cd CouponBondData/

/Users/Christian/Desktop/MSMFT/Semester 2/Fixed Income/MF728-RiskAnalysisProject/Data/CouponBonds/CouponBondData


In [ ]:
from DataOrganization import *
import datetime as dt

def combine_files() -> None:
    mainwd = Path.cwd()

    for paths in mainwd.iterdir():
        with open_folder(paths):
            folder_path = Path.cwd()

            price_df = pd.DataFrame()
            yield_df = pd.DataFrame()

            for i, csv in enumerate(folder_path.iterdir()):
                df = pd.read_csv(csv)

                #creating the info dataframe
                df_info = df.head(8).dropna(axis=1, how='all').dropna(axis=0, how='all')  # Drop rows and columns that are completely empty
                df_info = df_info.transpose().reset_index(drop=True) # Set the first column as index
                df_info.columns = df_info.iloc[0]  # Set the first row as column names
                df_info = df_info.drop(df_info.index[0])  # Drop the first row which is now redundant   
                df_info = df_info.squeeze()  # Convert the DataFrame to a Series
                df_info.index.name = None
                df_info.name = df_info["Company"]
                
                df_info["Maturity Date"] = dt.datetime.strptime(df_info["Maturity Date"].split(" ")[0], "%Y-%m-%d")  # Remove spaces from the "Issue Date" column
                df_info["Issue Date"] = dt.datetime.strptime(df_info["Issue Date"].split(" ")[0], "%Y-%m-%d")  # Remove spaces from the "Issue Date" column
                df_info["Tenor (days)"] = (df_info["Maturity Date"] - df_info["Issue Date"]).days

                # Creating the price and yield dataframe
                df_price_yield = df.tail(-8)  # Drop columns that are completely empty
                df_price_yield.columns = list(df_price_yield.iloc[0]) # Set the first row as column names
                df_price_yield = df_price_yield.reset_index(drop=True)
                df_price_yield.drop(0, inplace=True)  # Drop the first row which is now redundant
                df_price_yield.set_index('Date', inplace=True)  # Set the first column as index

                #setting up the two dataframes
                yield_df_temp = pd.concat([df_info, df_price_yield["Mid YTM"]], ignore_index=False)
                price_df_temp = pd.concat([df_info, df_price_yield["Mid Price"]], ignore_index=False)

                #naming the columns
                yield_df_temp.name = yield_df_temp["CUSIP"]
                price_df_temp.name = price_df_temp["CUSIP"]

                # Changing to dataframes
                yield_df_temp = yield_df_temp.to_frame()
                price_df_temp = price_df_temp.to_frame()

                # Concatenate the dataframes
                if i == 0:
                    price_df = price_df_temp
                    yield_df = yield_df_temp
                else:
                    price_df = price_df.join(price_df_temp, how='inner', rsuffix=f"_{i}")
                    
                    yield_df = yield_df.join(yield_df_temp, how='inner', rsuffix=f"_{i}")

            # Dropping the CUSIP column
            # price_df.drop("CUSIP", axis=0, inplace=True)
            # yield_df.drop("CUSIP", axis=0, inplace=True)

            # Saving the dataframes
            # print(folder_path)
            # os.remove(folder_path / f"Price_{folder_path.name}.csv")
            # os.remove(folder_path / f"Yield_{folder_path.name}.csv")
combine_files()

/var/folders/49/1jq9sd950fg6397s7sm972380000gn/T/ipykernel_35000/2650899376.py:15: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv)
/var/folders/49/1jq9sd950fg6397s7sm972380000gn/T/ipykernel_35000/2650899376.py:15: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv)
